## 10.14 FE: Variables Elegidas + MIN MAX TEND - Weights (Base Andres)

1) Parametros Modelos: mejor de 200 corridas de BO (se mantiene para todas las prueba de FE).

2) Meses Entrenamiento: 12

3) Peridos WFV: 3

4) Con Decay / Weights

5) Features

* Correccion de Variables
* Nuevas Variables con Visa y Master
* Calculo MIN, MAX y Tendencia de Todas (con CPP)
* 
'mv_status06','mcaja_ahorro','cpayroll_trx','cliente_antiguedad','mcuentas_saldo','mv_Finiciomora','mcuenta_corriente','mrentabilidad','ctrx_quarter','Visa_Finiciomora','mdescubierto_preacordado','cproductos','mactivos_margen','mprestamos_personales','Visa_status','mpayroll','Visa_mpagominimo','mv_cconsumos','mv_status07','mpasivos_margen','mv_mpagominimo','ctarjeta_visa_transacciones','ccomisiones_mantenimiento','ctarjeta_master','Visa_delinquency','mcomisiones_mantenimiento','mcomisiones_otras','mvr_mpagominimo','mtarjeta_visa_consumo','cliente_edad','cprestamos_personales','mttarjeta_visa_debitos_automaticos','Visa_mfinanciacion_limite','mrentabilidad_annual','mv_mpagospesos','ccuenta_debitos_automaticos','ccaja_seguridad','mv_msaldopesos','ctarjeta_visa_debitos_automaticos','mv_status04','Master_mpagospesos','mcuenta_debitos_automaticos','internet','ctarjeta_debito_transacciones','mv_status05','mv_status01','Visa_mpagospesos','mvr_mpagospesos','mcomisiones','mtransferencias_emitidas','tcallcenter','tcuentas','mvr_msaldopesos','Master_mfinanciacion_limite','Visa_msaldototal','ctarjeta_visa','mv_fechaalta','mvr_msaldototal','ccaja_ahorro','mcheques_emitidos_rechazados','cpagomiscuentas','mv_mlimitecompra','Visa_msaldopesos','mplazo_fijo_dolares','mv_status02','ctarjeta_master_transacciones','mv_mconsumototal','thomebanking','ccheques_emitidos_rechazados','mv_msaldototal','Visa_fechaalta','Master_delinquency','mv_status03','ccajas_consultas','Visa_cconsumos','Visa_mlimitecompra','mcaja_ahorro_dolares','ccallcenter_transacciones','Master_mpagominimo',

6) Genero PDF con Feature Importance de LGBM

### Librerias usadas por el programa

In [1]:
#limpio la memoria
rm(list=ls())
gc()

library("data.table")
library("ROCR")
library("lightgbm")
library("caret")
library("DiceKriging")
library("mlrMBO")
library("Rcpp")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,530915,28.4,1189710,63.6,654380,35.0
Vcells,1019481,7.8,8388608,64.0,1771249,13.6


Loading required package: R6

Loading required package: lattice

Loading required package: ggplot2

Loading required package: mlr

Loading required package: ParamHelpers

'mlr' is in maintenance mode since July 2019. Future development
efforts will go into its successor 'mlr3' (<https://mlr3.mlr-org.com>).


Attaching package: ‘mlr’


The following object is masked from ‘package:caret’:

    train


The following object is masked from ‘package:ROCR’:

    performance


Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames


Warning message:
“no DISPLAY variable so Tk is not available”


## Parámetros generales

In [2]:
#Archivo con datos totales (etiquetados hasta 201912 + parciales 202001 + sin etiquetar 202002)
karchivo_entrada      <-  "~/buckets/b1/datasetsOri/paquete_premium_faseII.txt.gz"

#Separador de campos en archivos
kcampos_separador     <-  "\t"

kdir_trabajo <- "~/buckets/b1/work/"

kdir_salida <- "~/buckets/b1/work/Fase II/"

#Campo que identifica las muestras
kcampo_id             <-  "numero_de_cliente"

#Campo que contiene la clase a estimar
kclase_nomcampo       <-  "clase_ternaria"

#Valor de interés
kclase_valor_positivo <-  "BAJA+2"

#Campos a borrar para el entrenamiento
kcampos_a_borrar      <-  c(kclase_nomcampo)

#Campo que contendrá a la variable objetivo generada
kobjetivo             <-  "clase01"

#Identificación del modelo
kmodelo               <-  "10.14 FE-VarsElegidas"

problema <- list()
problema$prob_corte    <-  0.025
problema$ganancia_TP   <-  29250 
problema$ganancia_FP   <-   -750

offset <- 0

In [3]:
#cargo TODOS los datos
dataset <- fread(karchivo_entrada)

In [4]:
#Genero lista ordenada de meses
meses <- sort(unique(dataset$foto_mes))

## Corrección de problemas puntuales en determinados meses

In [5]:
#acomodo los errores del dataset
dataset[ foto_mes==201701,  ccajas_consultas   := NA ]
dataset[ foto_mes==201702,  ccajas_consultas   := NA ]

dataset[ foto_mes==201801,  internet   := NA ]
dataset[ foto_mes==201801,  thomebanking   := NA ]
dataset[ foto_mes==201801,  chomebanking_transacciones   := NA ]
dataset[ foto_mes==201801,  tcallcenter   := NA ]
dataset[ foto_mes==201801,  ccallcenter_transacciones   := NA ]
dataset[ foto_mes==201801,  cprestamos_personales   := NA ]
dataset[ foto_mes==201801,  mprestamos_personales   := NA ]
dataset[ foto_mes==201801,  mprestamos_hipotecarios  := NA ]
dataset[ foto_mes==201801,  ccajas_transacciones   := NA ]
dataset[ foto_mes==201801,  ccajas_consultas   := NA ]
dataset[ foto_mes==201801,  ccajas_depositos   := NA ]
dataset[ foto_mes==201801,  ccajas_extracciones   := NA ]
dataset[ foto_mes==201801,  ccajas_otras   := NA ]

dataset[ foto_mes==201806,  tcallcenter   :=  NA ]
dataset[ foto_mes==201806,  ccallcenter_transacciones   :=  NA ]

dataset[ foto_mes==201904,  ctarjeta_visa_debitos_automaticos  :=  NA ]
dataset[ foto_mes==201904,  mttarjeta_visa_debitos_automaticos  := NA ]

dataset[ foto_mes==201905,  mrentabilidad     := NA ]
dataset[ foto_mes==201905,  mrentabilidad_anual     := NA ]
dataset[ foto_mes==201905,  mcomisiones       := NA ]
dataset[ foto_mes==201905,  mpasivos_margen  := NA ]
dataset[ foto_mes==201905,  mactivos_margen  := NA ]
dataset[ foto_mes==201905,  ctarjeta_visa_debitos_automaticos  := NA ]
dataset[ foto_mes==201905,  ccomisiones_otras := NA ]
dataset[ foto_mes==201905,  mcomisiones_otras := NA ]

dataset[ foto_mes==201910,  mpasivos_margen  := NA ]
dataset[ foto_mes==201910,  mactivos_margen  := NA ]
dataset[ foto_mes==201910,  ccomisiones_otras := NA ]
dataset[ foto_mes==201910,  mcomisiones_otras := NA ]
dataset[ foto_mes==201910,  mcomisiones       := NA ]
dataset[ foto_mes==201910,  mrentabilidad     := NA ]
dataset[ foto_mes==201910,  mrentabilidad_annual     := NA ]
dataset[ foto_mes==201910,  chomebanking_transacciones   := NA ]
dataset[ foto_mes==201910,  ctarjeta_visa_descuentos   := NA ]
dataset[ foto_mes==201910,  ctarjeta_master_descuentos   := NA ]
dataset[ foto_mes==201910,  mtarjeta_visa_descuentos   := NA ]
dataset[ foto_mes==201910,  mtarjeta_master_descuentos    := NA ]
dataset[ foto_mes==201910,  ccajeros_propios_descuentos   := NA ]
dataset[ foto_mes==201910,  mcajeros_propios_descuentos   := NA ]

dataset[ foto_mes==202001,  cliente_vip   := NA ]

### Esta seccion esta como texto. Es la que hace el ranking para intentar acomodar las distribuciones de variables. En caso de querer usarla se cambia la celda a codigo
fdividir_maximo  <- function( pvector )
{
  maximo <-  max( pvector, na.rm=TRUE )
  return(  pvector / maximo )
}

camposMonto <-  colnames(dataset)[  like( colnames(dataset), "^m|^Visa_m|^Master_m" )]

dataset[ , (camposMonto) := lapply( .SD, frankv,  na.last="keep", ties.method="dense" ) , by=foto_mes, .SDcols= camposMonto ]
dataset[ , (camposMonto) := lapply( .SD, fdividir_maximo ) , by=foto_mes, .SDcols= camposMonto ]

In [6]:
#Genera lista de meses
meses <- sort(unique(dataset$foto_mes))

In [7]:
#Verifico meses cargados
meses

[1] 201701 201702 201703 201704 201705 201706 201707 201708 201709 201710
[11] 201711 201712 201801 201802 201803 201804 201805 201806 201807 201808
[21] 201809 201810 201811 201812 201901 201902 201903 201904 201905 201906
[31] 201907 201908 201909 201910 201911 201912 202001 202002

## Feature Engineering
### Creacion de variables dentro de cada mes

In [8]:
#Guardo una lista de campos originales del Dataset
columnas_originales <-  copy(colnames( dataset ))

#INICIO de la seccion donde se deben hacer cambios con variables nuevas
#se crean los nuevos campos para MasterCard  y Visa, teniendo en cuenta los NA's
#varias formas de combinar Visa_status y Master_status
dataset[ , mv_status01       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
dataset[ , mv_status02       := Master_status +  Visa_status ]
dataset[ , mv_status03       := pmax( ifelse( is.na(Master_status), 9, Master_status) , ifelse( is.na(Visa_status), 9, Visa_status) ) ]
dataset[ , mv_status04       := ifelse( is.na(Master_status), 9, Master_status)  +  ifelse( is.na(Visa_status), 9, Visa_status)  ]
dataset[ , mv_status05       := ifelse( is.na(Master_status), 9, Master_status)  +  10*ifelse( is.na(Visa_status), 9, Visa_status)  ]

dataset[ , mv_status06       := ifelse( is.na(Visa_status), 
                                        ifelse( is.na(Master_status), 9, Master_status), 
                                        Visa_status)  ]

dataset[ , mv_status07       := ifelse( is.na(Master_status), 
                                        ifelse( is.na(Visa_status), 9, Visa_status), 
                                        Master_status)  ]


#combino MasterCard y Visa
dataset[ , mv_mfinanciacion_limite := rowSums( cbind( Master_mfinanciacion_limite,  Visa_mfinanciacion_limite) , na.rm=TRUE ) ]

dataset[ , mv_Fvencimiento         := pmin( Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE) ]
dataset[ , mv_Finiciomora          := pmin( Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE) ]
dataset[ , mv_msaldototal          := rowSums( cbind( Master_msaldototal,  Visa_msaldototal) , na.rm=TRUE ) ]
dataset[ , mv_msaldopesos          := rowSums( cbind( Master_msaldopesos,  Visa_msaldopesos) , na.rm=TRUE ) ]
dataset[ , mv_msaldodolares        := rowSums( cbind( Master_msaldodolares,  Visa_msaldodolares) , na.rm=TRUE ) ]
dataset[ , mv_mconsumospesos       := rowSums( cbind( Master_mconsumospesos,  Visa_mconsumospesos) , na.rm=TRUE ) ]
dataset[ , mv_mconsumosdolares     := rowSums( cbind( Master_mconsumosdolares,  Visa_mconsumosdolares) , na.rm=TRUE ) ]
dataset[ , mv_mlimitecompra        := rowSums( cbind( Master_mlimitecompra,  Visa_mlimitecompra) , na.rm=TRUE ) ]
dataset[ , mv_madelantopesos       := rowSums( cbind( Master_madelantopesos,  Visa_madelantopesos) , na.rm=TRUE ) ]
dataset[ , mv_madelantodolares     := rowSums( cbind( Master_madelantodolares,  Visa_madelantodolares) , na.rm=TRUE ) ]
dataset[ , mv_fultimo_cierre       := pmax( Master_fultimo_cierre, Visa_fultimo_cierre, na.rm = TRUE) ]
dataset[ , mv_mpagado              := rowSums( cbind( Master_mpagado,  Visa_mpagado) , na.rm=TRUE ) ]
dataset[ , mv_mpagospesos          := rowSums( cbind( Master_mpagospesos,  Visa_mpagospesos) , na.rm=TRUE ) ]
dataset[ , mv_mpagosdolares        := rowSums( cbind( Master_mpagosdolares,  Visa_mpagosdolares) , na.rm=TRUE ) ]
dataset[ , mv_fechaalta            := pmax( Master_fechaalta, Visa_fechaalta, na.rm = TRUE) ]
dataset[ , mv_mconsumototal        := rowSums( cbind( Master_mconsumototal,  Visa_mconsumototal) , na.rm=TRUE ) ]
dataset[ , mv_cconsumos            := rowSums( cbind( Master_cconsumos,  Visa_cconsumos) , na.rm=TRUE ) ]
dataset[ , mv_cadelantosefectivo   := rowSums( cbind( Master_cadelantosefectivo,  Visa_cadelantosefectivo) , na.rm=TRUE ) ]
dataset[ , mv_mpagominimo          := rowSums( cbind( Master_mpagominimo,  Visa_mpagominimo) , na.rm=TRUE ) ]

#a partir de aqui juego con la suma de Mastercard y Visa
dataset[ , mvr_Master_mlimitecompra:= Master_mlimitecompra / mv_mlimitecompra ]
dataset[ , mvr_Visa_mlimitecompra  := Visa_mlimitecompra / mv_mlimitecompra ]
dataset[ , mvr_msaldototal         := mv_msaldototal / mv_mlimitecompra ]
dataset[ , mvr_msaldopesos         := mv_msaldopesos / mv_mlimitecompra ]
dataset[ , mvr_msaldopesos2        := mv_msaldopesos / mv_msaldototal ]
dataset[ , mvr_msaldodolares       := mv_msaldodolares / mv_mlimitecompra ]
dataset[ , mvr_msaldodolares2      := mv_msaldodolares / mv_msaldototal ]
dataset[ , mvr_mconsumospesos      := mv_mconsumospesos / mv_mlimitecompra ]
dataset[ , mvr_mconsumosdolares    := mv_mconsumosdolares / mv_mlimitecompra ]
dataset[ , mvr_madelantopesos      := mv_madelantopesos / mv_mlimitecompra ]
dataset[ , mvr_madelantodolares    := mv_madelantodolares / mv_mlimitecompra ]
dataset[ , mvr_mpagado             := mv_mpagado / mv_mlimitecompra ]
dataset[ , mvr_mpagospesos         := mv_mpagospesos / mv_mlimitecompra ]
dataset[ , mvr_mpagosdolares       := mv_mpagosdolares / mv_mlimitecompra ]
dataset[ , mvr_mconsumototal       := mv_mconsumototal  / mv_mlimitecompra ]
dataset[ , mvr_mpagominimo         := mv_mpagominimo  / mv_mlimitecompra ]

columnas_extendidas <-  copy( setdiff(  colnames(dataset), columnas_originales ) )

### Generación de Variables entre meses

* Funcion CPP que genera tendencia, minimos y maximos

In [9]:
#------------------------------------------------------------------------------
#se calculan para los 6 meses previos el minimo, maximo y tendencia calculada con cuadrados minimos
#la formual de calculo de la tendencia puede verse en https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
#para la maxíma velocidad esta funcion esta escrita en lenguaje C, y no en la porqueria de R o Python

cppFunction('NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde ) 
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 4*n );

  for(int i = 0; i < n; i++)
  {

    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) ) 
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      { 
        xsum  += x[h] ;
        ysum  += y[h] ; 
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ; 
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}')

* Cálculo de Min, Max y Tendencia en ventana movild de 6 meses

In [10]:
#------------------------------------------------------------------------------
#Aqui comienza la creación de variables historicas

#ordeno por  numero_de_cliente y foto_mes
setorder( dataset,  numero_de_cliente, foto_mes )

#Esta es la cantidad de meses que utilizo para la historia
ventana_regresion  <- 6

last <- nrow( dataset )
kcampo_id_idx  <-  match( "numero_de_cliente", names(dataset) )
#----------


#creo el vector_desde que indica cada ventana
#de esta forma se acelera el procesamiento ya que lo hago una sola vez

vector_ids   <- dataset[[  kcampo_id_idx  ]]

vector_desde <- seq( -ventana_regresion+2,  nrow(dataset)-ventana_regresion+1 )
vector_desde[ 1:ventana_regresion ]  <-  1

for( i in 2:last )  if( vector_ids[ i-1 ] !=  vector_ids[ i ] ) {  vector_desde[i] <-  i }
for( i in 2:last )  if( vector_desde[i] < vector_desde[i-1] )  {  vector_desde[i] <-  vector_desde[i-1] }


columnas_no_procesar  <- c( "numero_de_cliente", "foto_mes", "clase_ternaria" )

#agrego al dataset las TENDENCIAS
#columnas_originales_a_procesar  <- setdiff( columnas_originales,  columnas_no_procesar  )  
columnas_originales_a_procesar  <- c('mv_status06','mcaja_ahorro','cpayroll_trx','cliente_antiguedad','mcuentas_saldo',
                                     'mv_Finiciomora','mcuenta_corriente','mrentabilidad','ctrx_quarter','Visa_Finiciomora',
                                     'mdescubierto_preacordado','cproductos','mactivos_margen','mprestamos_personales','Visa_status',
                                     'mpayroll','Visa_mpagominimo','mv_cconsumos','mv_status07','mpasivos_margen','mv_mpagominimo',
                                     'ctarjeta_visa_transacciones','ccomisiones_mantenimiento','ctarjeta_master','Visa_delinquency',
                                     'mcomisiones_mantenimiento','mcomisiones_otras','mvr_mpagominimo','mtarjeta_visa_consumo',
                                     'cliente_edad','cprestamos_personales','mttarjeta_visa_debitos_automaticos','Visa_mfinanciacion_limite',
                                     'mrentabilidad_annual','mv_mpagospesos','ccuenta_debitos_automaticos','ccaja_seguridad',
                                     'mv_msaldopesos','ctarjeta_visa_debitos_automaticos','mv_status04','Master_mpagospesos',
                                     'mcuenta_debitos_automaticos','internet','ctarjeta_debito_transacciones','mv_status05','mv_status01',
                                     'Visa_mpagospesos','mvr_mpagospesos','mcomisiones','mtransferencias_emitidas','tcallcenter',
                                     'tcuentas','mvr_msaldopesos','Master_mfinanciacion_limite','Visa_msaldototal','ctarjeta_visa',
                                     'mv_fechaalta','mvr_msaldototal','ccaja_ahorro','mcheques_emitidos_rechazados','cpagomiscuentas',
                                     'mv_mlimitecompra','Visa_msaldopesos','mplazo_fijo_dolares','mv_status02','ctarjeta_master_transacciones',
                                     'mv_mconsumototal','thomebanking','ccheques_emitidos_rechazados','mv_msaldototal','Visa_fechaalta',
                                     'Master_delinquency','mv_status03','ccajas_consultas','Visa_cconsumos','Visa_mlimitecompra',
                                     'mcaja_ahorro_dolares','ccallcenter_transacciones','Master_mpagominimo')

for(  campo  in  columnas_originales_a_procesar )
{
   campo_idx     <-   match( campo,  names(dataset) )
   col_original  <-   dataset[[  campo_idx  ]]

   nueva_col     <- fhistC( col_original, vector_desde ) 

   #agrego las nuevas columnas al dataset
   dataset[ , paste( campo, "__tend", sep="" ):= nueva_col[ (0*last +1):(1*last) ]  ]
   dataset[ , paste( campo, "__min" , sep="" ):= nueva_col[ (1*last +1):(2*last) ]  ]
   dataset[ , paste( campo, "__max" , sep="" ):= nueva_col[ (2*last +1):(3*last) ]  ]
   
   #Por ahora, no agrego el promedio
   #dataset[ , paste( campo, "__avg" , sep="" ):= nueva_col[ (3*last +1):(4*last) ]  ]
}


#dejo la clase como ultimo campo
nuevo_orden <-  c( setdiff( colnames( dataset ) , "clase_ternaria" ) , "clase_ternaria" )
setcolorder( dataset, nuevo_orden )


#ahora agrego tendencias TAMBIEN  para las variables nuevas

columnas_extendidas_a_procesar  <- setdiff( columnas_extendidas,  columnas_no_procesar  )  

for(  campo  in  columnas_extendidas_a_procesar )
{
   campo_idx     <-   match( campo,  names(dataset) )
   col_original  <-   dataset[[  campo_idx  ]]

   nueva_col     <- fhistC( col_original, vector_desde ) 

   #agrego las nuevas columnas al dataset
   dataset[ , paste( campo, "__tend", sep="" ):= nueva_col[ (0*last +1):(1*last) ]  ]
   dataset[ , paste( campo, "__min" , sep="" ):= nueva_col[ (1*last +1):(2*last) ]  ]
   dataset[ , paste( campo, "__max" , sep="" ):= nueva_col[ (2*last +1):(3*last) ]  ]
   
   #Por ahora, no agrego el promedio
   #dataset[ , paste( campo, "__avg" , sep="" ):= nueva_col[ (3*last +1):(4*last) ]  ]
}


#dejo la clase como ultimo campo
nuevo_orden <-  c( setdiff( colnames( dataset ) , "clase_ternaria" ) , "clase_ternaria" )
setcolorder( dataset, nuevo_orden )




In [11]:
#Genero la clase
dataset[,(kobjetivo) := as.integer(dataset[,..kclase_nomcampo] == kclase_valor_positivo)]

#Limpio campos a borrar
dataset[ ,  (kcampos_a_borrar) := NULL    ] 

Warning message in `[.data.table`(dataset, , ..kclase_nomcampo):
“Both 'kclase_nomcampo' and '..kclase_nomcampo' exist in calling scope. Please remove the '..kclase_nomcampo' variable in calling scope for clarity.”


### Cálculo de ganancia

In [12]:

fmetrica_ganancia  <- function( preds, data){
    
    clases <- getinfo(data, "label")
 
    gan <- as.numeric(sum(    (preds > problema$prob_corte + offset ) * 
                   ifelse( clases== 1, problema$ganancia_TP, problema$ganancia_FP )   
              ))/length(clases)*200000
         
  return(list(name = "ganancia", value =  gan, higher_better = TRUE))
     
}

### Esta seccion se puede hacer para reducir el dataset. 
Tener en cuenta que debe tener los 2 meses finales (202001+202002) mas todos los meses que se entrenen en walk forward considerando la cantidad de meses en cada entrenamiento.
**Ejemplo: Se hace Walk forward con Diciembre 2020 + Octubre 2020 + Agosto 2020 y se entrena sobre 12 meses necesito conservar 19 meses**
* 2 Meses finales Enero + Febrero 2020
* 5 Meses para cubrir hasta el primer walk forward Agosto a Diciembre 2019
* 12 meses de entrenamiento Agosto 2018 a Julio 2019

In [13]:
#Reduzco el dataset
meses_a_usar <- 19
total_meses <- length(meses)
mes_corte <- meses[total_meses-meses_a_usar]
dataset <- dataset[foto_mes>= mes_corte]

#redeterino lista de meses
meses <- sort(unique(dataset$foto_mes))
total_meses <- length(meses)


### Prueba Walk Forward Validation

In [14]:
#Determino total de meses
total_meses <- length(meses)

#Defino sobre que meses (offset) voy a hacer el WFV S recomienda fedinirno y NO TOCARLO MAS para que los resultados sean comparables
offsets_wfv <- c(-2,-4,-6)

for ( meswfv in total_meses + offsets_wfv){

mes_test <- meses[meswfv]
mes_hasta <- meses[(meswfv-2)]
mes_desde <- meses[(meswfv-1-12)] #ATENCION!!! Resta 12 porque entreno en HASTA 12 meses. Si se amplia el espacio hay que modificar ese numero por el maximo a usar
print(paste("Entreno de",mes_desde,"a",mes_hasta,"y testeo en",mes_test))
}

[1] "Entreno de 201811 a 201910 y testeo en 201912"
[1] "Entreno de 201809 a 201908 y testeo en 201910"
[1] "Entreno de 201807 a 201906 y testeo en 201908"


### Cargo los parametros

In [15]:
best_p = list()

best_p$bagging_fraction    = 0.245931249459893
best_p$feature_fraction    = 0.231407803166092
best_p$learning_rate       = 0.143584188569781
best_p$min_child_weight    = 0.0108096996449808
best_p$min_data_in_leaf    = 761
best_p$min_gain_to_split   = 0.21034602746677
best_p$max_depth           = 8
best_p$lambda_l1           = 0.70127235890829
best_p$lambda_l2           = 98.1357357243454
best_p$max_bin             = 32
best_p$num_leaves          = 324
best_p$q_meses             = 12
best_p$offset              = -0.00120733421194171
best_p$decay               = 0.969219937320735
best_p$iters               = 328.666666666667

best_p

$bagging_fraction
[1] 0.2459312

$feature_fraction
[1] 0.2314078

$learning_rate
[1] 0.1435842

$min_child_weight
[1] 0.0108097

$min_data_in_leaf
[1] 761

$min_gain_to_split
[1] 0.210346

$max_depth
[1] 8

$lambda_l1
[1] 0.7012724

$lambda_l2
[1] 98.13574

$max_bin
[1] 32

$num_leaves
[1] 324

$q_meses
[1] 12

$offset
[1] -0.001207334

$decay
[1] 0.9692199

$iters
[1] 328.6667

### Hago WFV y Obtengo los Resultados 

In [16]:
t0   <-  Sys.time()

# Determina decay desde el último mes de la lista (ajustar el 0.95 como hiperparametro)
mapa_pesos = data.table(foto_mes = meses, weight= best_p$decay^order(-meses))

#Determina una lista de pesos coincidente con el dataset
pesos = merge(dataset[,c("foto_mes")],mapa_pesos,on=foto_mes,sort=FALSE)[,weight]

#Inicializo totales de gnancia, auc e iteraciones
gan_add = 0
auc_add = 0
best_iter_add = 0

#determino total de meses
total_meses <- length(meses)

for ( meswfv in total_meses + offsets_wfv){

    mes_test <- meses[meswfv]
    mes_hasta <- meses[(meswfv-2)]
    mes_desde <- meses[(meswfv-1-best_p$q_meses)]

    weights <- pesos[dataset$foto_mes >= mes_desde & dataset$foto_mes <= mes_hasta]

    train <- dataset[foto_mes >= mes_desde & foto_mes <= mes_hasta]
    y.train <- as.matrix(train[,..kobjetivo])
    X.train <- as.matrix(train[,-..kobjetivo])

    test <- dataset[foto_mes == mes_test]
    y.test <- as.matrix(test[,..kobjetivo])
    X.test <- as.matrix(test[,-..kobjetivo])
    
    # Limpio Memoria
    rm(train, test)

    #genero el formato requerido por LightGBM
    dtrain  <-   lgb.Dataset( data  = X.train,
                                    label = y.train, 
                                    free_raw_data=FALSE,
                                    weight=weights
                                    )

    #genero el formato requerido por LightGBM
    dtest  <-   lgb.Dataset( data  = X.test,
                                    label = y.test, 
                                    free_raw_data=FALSE
                                    )    

    # Limpio Memoria
    rm(y.train, X.train, y.test, X.test)
    
    offset <- best_p$offset

    modelo = lgb.train( 
                    data = dtrain,
                    objective = "binary",
                    nrounds= 5000,
                    eval=fmetrica_ganancia,
                    valids=list(test=dtest),
                    metric=c("ganancia", "auc"),
                    early_stopping_rounds=100,
                    boost_from_average=TRUE,
                    bagging_fraction=best_p$bagging_fraction, 
                    feature_fraction=best_p$feature_fraction, 
                    learning_rate=best_p$learning_rate,  
                    min_child_weight=best_p$min_child_weight, 
                    min_data_in_leaf=best_p$min_data_in_leaf, 
                    min_gain_to_split=best_p$min_gain_to_split, 
                    max_depth=best_p$max_depth, 
                    lambda_l1=best_p$lambda_l1, 
                    lambda_l2=best_p$lambda_l2, 
                    max_bin=best_p$max_bin, 
                    num_leaves=best_p$num_leaves 
                    )

    t1   <-  Sys.time()
    tiempo_corrida <-  as.numeric( t1 - t0, units = "secs")

    #Obtengo ganancia paso
    gan <- max(unlist(modelo$record_evals$test$ganancia$eval))
    best_iter <- which.max(unlist(modelo$record_evals$test$ganancia$eval))
    auc <- unlist(modelo$record_evals$test$auc$eval)[best_iter]

    # Guardo Resultados del WFV en CSV
    cat(kmodelo,
        paste0("Train: ",mes_desde,"-",mes_hasta," Test: ",mes_test),
        gan,
        paste0(auc,"\n"),
        file = "~/buckets/b1/work/fase2experimentos.csv",sep=",", fill=F, append = T)

    #Acumulo en totales de WFV
    gan_add = gan/length(offsets_wfv) + gan_add
    auc_add = auc/length(offsets_wfv) + auc_add
    best_iter_add = best_iter/length(offsets_wfv) + best_iter_add
    
    # Tabla de Variables Mas Importantes del Modelo
    tree_imp <- lgb.importance(modelo, percentage = TRUE)

    # Grafico de Variables Mas Importantes del Modelo
    pdf( paste0("~/buckets/b1/work/Fase II Importance/", kmodelo, "_" ,mes_test, ".pdf") )
    lgb.plot.importance(
      tree_imp,
      top_n = 50L,
      measure = "Gain",
      left_margin = 10L,
      cex = NULL
    )
    dev.off()    
    
    # Limpio Memoria
    rm(dtrain, dtest, modelo)
    # Reseteo Offset
    offset <- 0
}

# Guardo Resultados Promedios en CSV
cat(kmodelo,
    paste0("Promedio WFV"),
    gan_add,
    paste0(auc_add,"\n"),
    file = "~/buckets/b1/work/fase2experimentos.csv",sep=",", fill=F, append = T)

[LightGBM] [Info] Number of positive: 11379, number of negative: 2283392
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 1.009731 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13655
[LightGBM] [Info] Number of data points in the train set: 2294771, number of used features: 503
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.004882 -> initscore=-5.317381
[LightGBM] [Info] Start training from score -5.317381
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]:	test's auc:0.903986	test's ganancia:5.68465e+06 
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]:	test's auc:0.913455	test's ganancia:5.68465e+06 
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]:	test's auc:0.918521	test's ganancia:7.6972e+06 
[LightGBM] [Warning] No further splits with pos

In [17]:
# Al final de WFK imprimo el Promedio
print(paste("Ganancia Promedio",gan_add,"AUC Promedio",auc_add,"Mejor Iteracion Promedio",best_iter_add))

[1] "Ganancia Promedio 12156949.8669613 AUC Promedio 0.943253845696863 Mejor Iteracion Promedio 273"


## Genero predicción

### Genero dataset de entrenamiento

In [ ]:
t0   <-  Sys.time()
total_meses <- length(meses)

mes_predict <- meses[total_meses]
mes_hasta <- meses[(total_meses-2)]
mes_desde <- meses[(total_meses-1-best_p$q_meses)]

In [ ]:
# Determina decay desde el último mes de la lista (ajustar el 0.95 como hiperparametro)
mapa_pesos = data.table(foto_mes = meses, weight= best_p$decay^order(-meses))

#Determina una lista de pesos coincidente con el dataset
pesos = merge(dataset[,c("foto_mes")],mapa_pesos,on=foto_mes,sort=FALSE)[,weight]

#Filtro pesos por meses a usar en entrenamiento
weights <- pesos[dataset$foto_mes >= mes_desde & dataset$foto_mes <= mes_hasta]

#Filtro dataset de entrenamiento
train <- dataset[foto_mes >= mes_desde & foto_mes <= mes_hasta]
y.train <- as.matrix(train[,..kobjetivo])
X.train <- as.matrix(train[,-..kobjetivo])

#genero el formato requerido por LightGBM
dtrain  <-   lgb.Dataset( data  = X.train,
                               label = y.train, 
                               missing=NA,
                               free_raw_data=FALSE,
                               weight=weights
                              )

### Entreno modelo

In [ ]:
modelo = lgb.train( 
               data = dtrain,
               objective = "binary",
               nrounds= round(best_p$iters),
#               eval=fmetrica_ganancia,
#               metric=c("ganancia", "auc"),
#               early_stopping_rounds=100,
               boost_from_average=TRUE,
               bagging_fraction=best_p$bagging_fraction, 
               feature_fraction=best_p$feature_fraction, 
               learning_rate=best_p$learning_rate,  
               min_child_weight=best_p$min_child_weight, 
               min_data_in_leaf=best_p$min_data_in_leaf, 
               min_gain_to_split=best_p$min_gain_to_split,     
               max_depth=best_p$max_depth, 
               lambda_l1=best_p$lambda_l1, 
               lambda_l2=best_p$lambda_l2, 
               max_bin=best_p$max_bin, 
               num_leaves=best_p$num_leaves 
              )

t1   <-  Sys.time()
tiempo_corrida <-  as.numeric( t1 - t0, units = "secs")

### Genero predicción y grabo

In [ ]:
#cargo los datos
dpredict <- dataset[foto_mes == mes_predict]

#Armo dataset de salida
dataset_salida <- dpredict[,..kcampo_id]
setnames(dataset_salida, c("Id") )

In [ ]:
#aplico el modelo a los datos sin clase, 202002
Preds  <- predict(modelo,as.matrix(dpredict[,-..kobjetivo]))

In [ ]:
#Selecciono los que estan por encima del umbral
dpredict$Predicted <- Preds > problema$prob_corte  + best_p$offset

#Grabo salida
dataset_salida[,"Predicted"] <- dpredict$Predicted
write.csv(dataset_salida,paste0(kdir_salida, kmodelo, "_salida.csv"), row.names = FALSE)

In [ ]:
#Minimo Score
min(Preds)

In [ ]:
#Maximo Sco
max(Preds)